In [15]:
import numpy as np
import pandas as pd
from scipy.io import loadmat, mmread
import sys
import itertools 

In [68]:
import rankingmethod
import rankingmeasure
import imp
imp.reload(rankingmethod)
imp.reload(rankingmeasure)

<module 'rankingmeasure' from '/home/george/Documents/diploma/code/rankingmeasure.py'>

In [86]:
def all_measures(train, test, method, skip_train=False, top=None):
    print("precision5", rankingmeasure.get_prec_K(5, train, test, 
            method, skip_train=skip_train, top=top))
    print("1-call5", rankingmeasure.get_one_recal_K(5, train, test, 
            method, skip_train=skip_train, top=top))
    print("MRR", rankingmeasure.get_MRR(train, test, 
            method, skip_train=skip_train, top=top))

In [18]:
def make_valid_data(data, thres=25, reindex=True):
    list_good_user = list(np.where(np.bincount(data[:, 0]) > thres)[0])
    set_good_user = set(list_good_user)
    epinion = []
    for i,j in data:
        if i in set_good_user:
            epinion.append([i, j])
    epinion = np.array(epinion)
    if reindex:
        for new_u, u in enumerate(np.sort(list_good_user)):
            epinion[epinion[:, 0] == u, 0] = new_u
        print(epinion.shape)
        list_good_item = np.unique(epinion[:, 1])
        for new_i, i in enumerate(np.sort(list_good_item)):
            epinion[epinion[:, 1] == i, 1] = new_i
    user_item = np.max(epinion, axis=0) + 1
    return epinion, user_item


def create_listarray(data):
    xlist = []
    for u in np.sort(np.unique(data[:, 0])):
        xlist.append(data[data[:, 0] == u, 1])
    return xlist

In [19]:
def givenK_train_test(data, K):
    xlist = create_listarray(data)
    
    train = []
    test = []
    for x in xlist:
        train_ind = np.random.choice(x.shape[0], K, replace=False)
        tmp = np.ones(x.shape[0])
        tmp[train_ind] = 0
        test_ind = np.arange(x.shape[0])[tmp == 1]
        
        train.append(x[train_ind])
        test.append(x[test_ind])
        
    return (train, test, xlist)

##Мои данные

In [196]:
data = pd.read_csv("../datasets/soc-Epinions1.txt", sep='\t', header=4)
data = data.as_matrix()

In [303]:
#создание epinion удалив от туда пользователей, у которых количество отзывов меньше 25
#Также проводится переиндексанция пользователей и предметов

In [ ]:
epinion = make_valid_data(data)

In [ ]:
train, test, traintest = givenK_train_test(epinion, 5)

In [ ]:
poprec = rankingmethod.PopRec(user_item)
poprec.fit(train)
top = poprec.get_list(0)[:3]
all_measures(train, test, poprec, True, top)
all_measures(train, test, poprec, True)

In [345]:
list_reg = [0.01, 0.1 ,0.5]
list_lrate = [0.001, 0.005, 0.01, 0.05, 0.1 ]
list_K = [10, 5, 7, 12, 15]

for K in list_K:
    for lrate in list_lrate:
        for reg in list_reg:
            climf = rankingmethod.CLiMF(user_item,K=K, reg=reg, lrate=lrate, verbose=0, maxiter=100)
            climf.fit(train)
            print(K, lrate, reg)
            print(rankingmeasure.get_MRR(train, test, 
            climf, skip_train=False, top=top))

10 0.001 0.01
0.0068061377633100405
10 0.001 0.1
0.008031961359121426
10 0.001 0.5
0.005912269091311202
10 0.005 0.01
0.007815140434096543
10 0.005 0.1
0.00652022706541012
10 0.005 0.5
0.006532552664074765
10 0.01 0.01
0.007028481810001253
10 0.01 0.1
0.005199687139427464
10 0.01 0.5
0.00642130218487009
10 0.05 0.01
0.007314500601830544
10 0.05 0.1
0.00563555140338767
10 0.05 0.5
0.005847528094546522
10 0.1 0.01
0.008851060117673922
10 0.1 0.1
0.008597647640724496
10 0.1 0.5
0.007773293622440457
5 0.001 0.01
0.008755247464899283
5 0.001 0.1
0.010599960371667312
5 0.001 0.5
0.006476902717603026
5 0.005 0.01
0.010910411353978392
5 0.005 0.1
0.005833571238977291
5 0.005 0.5
0.008890794054437025
5 0.01 0.01
0.015242684443990519
5 0.01 0.1
0.006276795759708992
5 0.01 0.5
0.005705023945597651
5 0.05 0.01
0.022483016113061575
5 0.05 0.1
0.005442549602997411
5 0.05 0.5
0.008115720609231233
5 0.1 0.01
0.019875807579590635
5 0.1 0.1
0.009115952585898184
5 0.1 0.5
0.01036981495442007
7 0.001 0.01

KeyboardInterrupt: 

##Данные создателей CLiMF

In [20]:
tmp = mmread("../datasets/epinions_original/EP25_UPL5_test.mtx")
test = np.array(list(zip(tmp.row, tmp.col)))
tmp = mmread("../datasets/epinions_original/EP25_UPL5_train.mtx")
train = np.array(list(zip(tmp.row, tmp.col)))
user_item = [max(np.max(test[:,0]), np.max(train[:, 0])) + 1, max(np.max(test[:,1]), np.max(train[:,1])) + 1]

In [21]:
train = create_listarray(train)
test  = create_listarray(test)

In [77]:
randomrec = rankingmethod.RandomRec(user_item)
randomrec.fit(train)
all_measures(train, test, randomrec, False)

precision5 0.00148367952522
1-call5 0.00135650699449
MRR 0.009444793407610922


In [22]:
poprec = rankingmethod.PopRec(user_item)
poprec.fit(train)
top = poprec.get_list(0)[:3]
all_measures(train, test, poprec, True, top)
all_measures(train, test, poprec, True)

precision5 0.0538787621874
1-call5 0.0538787621874
MRR 0.11273876816174451
precision5 0.171131835524
1-call5 0.171131835524
MRR 0.4020232337666698


In [36]:
rankingmeasure.get_AUC(train, test, poprec, True, K=user_item[1])

0.8445145971450787

In [50]:
rankingmeasure.get_NDCG(train, test, poprec, True, K = 5)

0.19231475614860422

In [51]:
rankingmeasure.get_NDCG(train, test, poprec, True, K = 3)

0.21634665392029417

In [40]:
1 / np.log(np.arange(5) + 2)

array([ 1.44269504,  0.91023923,  0.72134752,  0.62133493,  0.55811063])

In [ ]:
climf = rankingmethod.CLiMF(user_item, maxiter=40, lrate = 0.1, verbose=2)
climf.fit(train)


In [106]:
all_measures(train, test, climf, False)

precision5 0.0635438745231
1-call5 0.0635438745231
MRR 0.16061628627384766


In [113]:
all_measures(train, test, climf, True, top)

precision5 0.0412038999576
1-call5 0.0412038999576
MRR 0.11461895836301948


In [123]:
list_reg = [0.001, 0.002, 0.005, 0.0005]
list_lrate = [0.05, 0.07, 0.1, 0.12, 0.15 ]
list_K = [10, 12, 15, 7, 5]
times_count = 3

for K, lrate, reg in itertools.product(list_K, list_lrate, list_reg):
    MRR = 0
    
    for t in range(times_count):
        climf = rankingmethod.CLiMF(user_item,K=K, reg=reg, lrate=lrate, verbose=0, maxiter=50)
        climf.fit(train)
        MRR += rankingmeasure.get_MRR(train, test, climf, skip_train=False)
    print("K", K, "lrate", lrate, "reg", reg, "MRR", MRR/times_count)
    sys.stdout.flush()

K 10 lrate 0.05 reg 0.001 MRR 0.16740641961680544
K 10 lrate 0.05 reg 0.002 MRR 0.1632928986028329
K 10 lrate 0.05 reg 0.005 MRR 0.1527730041694714
K 10 lrate 0.05 reg 0.0005 MRR 0.1622970666170243
K 10 lrate 0.07 reg 0.001 MRR 0.1617021688446262
K 10 lrate 0.07 reg 0.002 MRR 0.15214149672638577
K 10 lrate 0.07 reg 0.005 MRR 0.1428759872783604
K 10 lrate 0.07 reg 0.0005 MRR 0.1558988328476956
K 10 lrate 0.1 reg 0.001 MRR 0.1609389294595767
K 10 lrate 0.1 reg 0.002 MRR 0.14313962174434888
K 10 lrate 0.1 reg 0.005 MRR 0.12702524191410772
K 10 lrate 0.1 reg 0.0005 MRR 0.1433087707886965
K 10 lrate 0.12 reg 0.001 MRR 0.14271491223696034
K 10 lrate 0.12 reg 0.002 MRR 0.13561491669555753
K 10 lrate 0.12 reg 0.005 MRR 0.12988356090059913
K 10 lrate 0.12 reg 0.0005 MRR 0.13972362078679446
K 10 lrate 0.15 reg 0.001 MRR 0.06756475308278055
K 10 lrate 0.15 reg 0.002 MRR 0.06772883536587512
K 10 lrate 0.15 reg 0.005 MRR 0.06770193334254045
K 10 lrate 0.15 reg 0.0005 MRR 0.06781537216288246
K 12 lr

In [56]:
bpr_mf = rankingmethod.BPR_MF(user_item, K = 20, lrate=0.001, regIpos=0.1, 
                              regIneg=0.1, regU=0.1, maxiter=30, verbose=2 )

In [57]:
bpr_mf.fit(train)

iteration 0 loss 168133.42168
iteration 1 loss 167956.654651
iteration 2 loss 167804.477401
iteration 3 loss 167670.597582
iteration 4 loss 167553.965622
iteration 5 loss 167439.293842
iteration 6 loss 167339.720497
iteration 7 loss 167235.848677
iteration 8 loss 167153.208666
iteration 9 loss 167067.838843
iteration 10 loss 166986.679627
iteration 11 loss 166907.217323
iteration 12 loss 166827.097698
iteration 13 loss 166758.289538
iteration 14 loss 166685.230926
iteration 15 loss 166612.21482
iteration 16 loss 166543.017349
iteration 17 loss 166469.918385
iteration 18 loss 166394.132114
iteration 19 loss 166315.785495
iteration 20 loss 166240.804473
iteration 21 loss 166166.851581
iteration 22 loss 166089.925949
iteration 23 loss 166015.110329
iteration 24 loss 165919.477892
iteration 25 loss 165840.434111
iteration 26 loss 165760.177043
iteration 27 loss 165663.653649
iteration 28 loss 165575.566196
iteration 29 loss 165468.6769


In [64]:
all_measures(train, test, bpr_mf, True)

precision5 0.171343789741
1-call5 0.171343789741
MRR 0.40536315042490445


In [65]:
rankingmeasure.get_AUC(train, test, bpr_mf, True, K=user_item[1])

0.83119180356683098

In [67]:
rankingmeasure.get_NDCG(train, test, bpr_mf, True, K=5)

0.19229051846946299

In [153]:
import rankingmethod
import rankingmeasure
import imp
imp.reload(rankingmethod)
imp.reload(rankingmeasure)

<module 'rankingmeasure' from '/home/george/Documents/diploma/code/rankingmeasure.py'>

In [154]:
imf = rankingmethod.iMF(user_item, K=20, lmbd=0.01, alpha=40, maxiter=30, verbose=2)

In [155]:
imf.fit(train)

iteration 0 loss 675321.80008
iteration 1 loss 615451.702336
iteration 2 loss 598136.450685
iteration 3 loss 588866.66702
iteration 4 loss 582948.327657
iteration 5 loss 578913.988029
iteration 6 loss 576065.078573
iteration 7 loss 573985.763889
iteration 8 loss 572417.568027
iteration 9 loss 571198.968984
iteration 10 loss 570227.235156
iteration 11 loss 569435.206968
iteration 12 loss 568777.635541
iteration 13 loss 568223.129565
iteration 14 loss 567749.314417
iteration 15 loss 567339.847649
iteration 16 loss 566982.530012
iteration 17 loss 566668.078287
iteration 18 loss 566389.307116
iteration 19 loss 566140.569568
iteration 20 loss 565917.365498
iteration 21 loss 565716.061729
iteration 22 loss 565533.688814
iteration 23 loss 565367.791578
iteration 24 loss 565216.318173
iteration 25 loss 565077.537083
iteration 26 loss 564949.974549
iteration 27 loss 564832.366966
iteration 28 loss 564723.624257
iteration 29 loss 564622.801311


In [156]:
all_measures(train, test, imf, True)

precision5 0.192963119966
1-call5 0.546417973717677
MRR 0.3797155253723866


In [157]:
all_measures(train, test, imf, True, top)

precision5 0.172742687579
1-call5 0.49936413734633317
MRR 0.32784987239112395


In [308]:
all_measures(train, test, bpr_mf, True, top)

precision5 0.0624841034337
1-call5 0.0624841034337
MRR 0.1279237836247043


In [304]:
all_measures(train, test, poprec, True)

precision5 0.171131835524
1-call5 0.171131835524
MRR 0.4020232337666698


In [307]:
all_measures(train, test, poprec, True, top)

precision5 0.0538787621874
1-call5 0.0538787621874
MRR 0.11273876816174451


In [1]:
from sklearn.metrics import roc_auc_score

In [11]:
a = np.array([False,False, True, True])
b = np.array([0,2,1,3])
print(roc_auc_score(a, b, average=None))

0.75


In [14]:
(a == True).any()

True

In [8]:
np.arange(5)[::-1]

array([4, 3, 2, 1, 0])